In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
CEREBRAS_API_KEY=os.getenv('CEREBRAS_API_KEY')
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_TRACING=os.getenv('LANGSMITH_TRACING')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_PROJECT=os.getenv('LANGSMITH_PROJECT')

In [22]:
os.environ['CEREBRAS_API_KEY'] = CEREBRAS_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['LANGSMITH_API_KEY'] = LANGSMITH_API_KEY
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGSMITH_PROJECT'] = LANGSMITH_PROJECT

In [11]:
from langchain_cerebras import ChatCerebras

llm = ChatCerebras(
    model="llama-3.3-70b",
)

In [25]:
while True:
    question = input("enter a prompt, to quit press q: ")
    if question != "quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye!")
        break

enter a prompt, to quit press q:  hello, respond in 5 words. How can i surpass the limit of being a human


Merge with artificial intelligent systems.


enter a prompt, to quit press q:  quit


goodbye!


In [26]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [27]:
store = {}

In [30]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [29]:
config = {"configurable": {"session_id": "firstchat"}}

In [32]:
model_with_memory = RunnableWithMessageHistory(llm, get_session_history)

In [33]:
model_with_memory.invoke(("Hi, i am Ochi Ojie"), config=config).content

"Hello Ochi Ojie, it's nice to meet you. Is there something I can help you with or would you like to chat?"

In [34]:
model_with_memory.invoke(("what is my name?"), config=config).content

'Your name is Ochi Ojie.'

In [35]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, i am Ochi Ojie', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Ochi Ojie, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 43, 'total_tokens': 72, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.3-70b', 'system_fingerprint': 'fp_a4e4dbcbd01d96252206', 'id': 'chatcmpl-9f5ea9fb-677f-4c81-a21a-6a082ef05611', 'finish_reason': 'stop', 'logprobs': None}, id='run-3edd5e83-a9b7-425d-a5f9-4852f779c57a-0', usage_metadata={'input_tokens': 43, 'output_tokens': 29, 'total_tokens': 72, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Ochi Ojie.', additional_kwargs={'refusal': None},

# RAG with LCEL

In [40]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [72]:
# from sentence_transformers import SentenceTransformer
# embedding = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

from langchain_huggingface import HuggingFaceEmbeddings

# Wrapping the SentenceTransformer with LangChain-compatible embedding
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

In [73]:
## Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls = TextLoader)
docs = loader.load()

### Creating chunks using recursive character splitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)

new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

### Creating retriever using vector DB

db = Chroma.from_documents(new_docs, embedding)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [74]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [75]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [67]:
question = "what is minoxidil ? can you tell me how it's useful to me?"
print(retrieval_chain.invoke(question))

Minoxidil is a topical medication primarily used. Unfortunately, the provided context does not specify its usefulness or how it can be useful to you. It only defines what Minoxidil is.


In [69]:
question = "what is minoxidil FDA-approved for ?"
print(retrieval_chain.invoke(question))

The provided context does not specify what Minoxidil is FDA-approved for. It only mentions that Minoxidil is FDA-approved, but the purpose or condition for the approval is not stated in the given context.


In [76]:
question = "what is llama3 ? can you highlight 3 important points"
print(retrieval_chain.invoke(question))

Based on the provided context, Llama 3 appears to be a version of a language model or AI system. Here are three important points about Llama 3:

1. **Release time frame mentioned but unclear**: The release of Llama 3 is mentioned, but the exact date is not consistently stated. One document mentions "is Llama 3 released in April 2024," suggesting a future release, while another mentions "February 2023," which could indicate an earlier version or something else entirely.
2. **Versions and updates**: Llama 3 has a specific version with "8B parameter," indicating that Llama 3 is developed with different capacities or configurations, possibly for different applications or improvements.
3. **Associated with Meta**: The release of Llama 3 is mentioned alongside actions taken by "Meta," suggesting that Meta, possibly Meta Platforms, Inc. (the company behind Facebook and Instagram), is involved in the development, release, or support of Llama 3.


# Tools and Agents

In [77]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [78]:
api_wrapper = WikipediaAPIWrapper()

In [79]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [81]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}